In [15]:
# https://wikidocs.net/24949

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# 1. Data

In [16]:
corpus = [
    "this is banana",
    "banana is long",
    "Is a banana a fruit or not",
    "i love pineapple"
]
vec = CountVectorizer()
vec.fit(corpus)

print(vec.vocabulary_)
print(vec.transform(corpus).toarray())

{'this': 8, 'is': 2, 'banana': 0, 'long': 3, 'fruit': 1, 'or': 6, 'not': 5, 'love': 4, 'pineapple': 7}
[[1 0 1 0 0 0 0 0 1]
 [1 0 1 1 0 0 0 0 0]
 [1 1 1 0 0 1 1 0 0]
 [0 0 0 0 1 0 0 1 0]]


In [17]:
A = vec.transform(corpus).toarray()
print('A (shape) :', A.shape)

A (shape) : (4, 9)


# 2. Full SVD

In [18]:
U, s, VT = np.linalg.svd(A, full_matrices = True)

print("U (shape) :", U.shape)
print("sigma (shape) :", s.shape)
print("VT (shape) :", VT.shape)

U (shape) : (4, 4)
sigma (shape) : (4,)
VT (shape) : (9, 9)


In [19]:
# 직교행렬 U
U.round(2)

array([[ 0.5 ,  0.5 ,  0.  ,  0.71],
       [ 0.5 ,  0.5 ,  0.  , -0.71],
       [ 0.71, -0.71,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ]])

In [20]:
# 시그마
# 특잇값 벡터
# 대각 행렬로 변환
S = np.zeros((4, 9))
S[:4, :4] = np.diag(s)

S.round(2)

array([[2.8 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.47, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.41, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [21]:
# 직교 행렬
VT.round(2)

array([[ 0.61,  0.25,  0.61,  0.18,  0.  ,  0.25,  0.25,  0.  ,  0.18],
       [ 0.2 , -0.48,  0.2 ,  0.34,  0.  , -0.48, -0.48,  0.  ,  0.34],
       [-0.  , -0.  , -0.  ,  0.  ,  0.71,  0.  ,  0.  ,  0.71,  0.  ],
       [-0.  , -0.  , -0.  , -0.71,  0.  ,  0.  ,  0.  ,  0.  ,  0.71],
       [-0.21, -0.01,  0.5 , -0.29,  0.5 , -0.14, -0.14, -0.5 , -0.29],
       [-0.19, -0.52,  0.14,  0.05,  0.  ,  0.79, -0.21,  0.  ,  0.05],
       [-0.19, -0.52,  0.14,  0.05,  0.  , -0.21,  0.79,  0.  ,  0.05],
       [-0.21, -0.01,  0.5 , -0.29, -0.5 , -0.14, -0.14,  0.5 , -0.29],
       [-0.65,  0.4 ,  0.23,  0.42,  0.  ,  0.01,  0.01,  0.  ,  0.42]])

In [22]:
# 원본 행렬 복구
A_hat = np.dot(np.dot(U, S), VT).round(2)

print("원본행렬과 동일 여부 :", np.allclose(A, A_hat))
A_hat

원본행렬과 동일 여부 : True


array([[ 1., -0.,  1., -0.,  0., -0., -0.,  0.,  1.],
       [ 1.,  0.,  1.,  1.,  0., -0., -0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  1.,  1.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.]])

# 3. Truncated SVD

In [23]:
size = 2

# 특잇값 대각 행렬인 시그마 축소
S = S[:size, :size]

S.round(2)

array([[2.8 , 0.  ],
       [0.  , 1.47]])

In [24]:
# 직교 행렬 U 축소
U = U[:, :size]

U.round(2)

array([[ 0.5 ,  0.5 ],
       [ 0.5 ,  0.5 ],
       [ 0.71, -0.71],
       [ 0.  ,  0.  ]])

In [25]:
# 전치 행렬 VT 축소
VT = VT[:size, :]

VT.round(2)

array([[ 0.61,  0.25,  0.61,  0.18,  0.  ,  0.25,  0.25,  0.  ,  0.18],
       [ 0.2 , -0.48,  0.2 ,  0.34,  0.  , -0.48, -0.48,  0.  ,  0.34]])

In [26]:
# 복구
A_prime = np.dot(np.dot(U, S), VT)
print(A)
print(A_prime.round(2))

[[1 0 1 0 0 0 0 0 1]
 [1 0 1 1 0 0 0 0 0]
 [1 1 1 0 0 1 1 0 0]
 [0 0 0 0 1 0 0 1 0]]
[[ 1.  -0.   1.   0.5  0.  -0.  -0.   0.   0.5]
 [ 1.   0.   1.   0.5  0.  -0.  -0.   0.   0.5]
 [ 1.   1.   1.   0.   0.   1.   1.   0.  -0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0. ]]


- 대체적으로 기존에 0인 값들은 0에 가까운 값이 나왔다.
- 제대로 복구되지 않은 구간도 존재한다.
- 해당 데이터는 문서의 topic count vectorize 데이터를 나타내던 행렬이다.
    - 즉, 행렬의 크기는 문서 수 X 토픽의 수
    - 이를 4 x 4 에서 4 x 2 로 줄여냈는데, 여기서 문서의 수는 유지되었지만, 토픽의 수는 줄어들었다.
    - 즉, 4개의 문서를 2개의 값으로 표현을 해내야 하는데, 이를 값으로 표현하면,
        - U (4 X 4 -> 4 X 2) : 잠재 의미를 표현하기 위한 각 각의 문서 벡터
        - VT (9 X 9 -> 2 X 9) : 잠재 의미를 표현하기 위한 각 각의 토픽 벡터
    -> 즉, 간단히 말하면 원래는 9개의 토픽으로 표현할 수 있던 문서 벡터를 2개의 토픽으로만 표현하여 잠재의미를 표현하는 것 이다.
    -> 위의 원본 행렬에서 마지막 문서벡터는 다른 문서 벡터와 완전히 반대되는 토픽을 가지고 있다. 
    -> 아래의 잠재의미 행렬에서 마지막 문서벡터는 복구되지 않았다. 행렬 분해에서는 행렬에서 강한 특징을 가지고 있는 값이 살아남는다.